#Setup Java and Spark

Get Java

In [1]:
!apt-get install openjdk-8-jdk-headless -qq 

Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading database ... 144628 files and directories currently installed.)
Preparing to unpack .../openjdk-8-jre-headless_8u272-b10-0ubuntu1~18.04_amd64.deb ...
Unpacking openjdk-8-jre-headless:amd64 (8u272-b10-0ubuntu1~18.04) ...
Selecting previously unselected package openjdk-8-jdk-headless:amd64.
Preparing to unpack .../openjdk-8-jdk-headless_8u272-b10-0ubuntu1~18.04_amd64.deb ...
Unpacking openjdk-8-jdk-headless:amd64 (8u272-b10-0ubuntu1~18.04) ...
Setting up openjdk-8-jre-headless:amd64 (8u272-b10-0ubuntu1~18.04) ...
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/orbd to provide /usr/bin/orbd (orbd) in auto mode
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/servertool to provide /usr/bin/servertool (servertool) in auto mode
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/tnameserv to provide /usr/bin/tnameserv (tnameserv) in auto mode
Setting up ope

Get Spark

In [2]:
!wget https://mirrors.estointernet.in/apache/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz 

--2020-11-04 10:29:57--  https://mirrors.estointernet.in/apache/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
Resolving mirrors.estointernet.in (mirrors.estointernet.in)... 43.255.166.254, 2403:8940:3:1::f
Connecting to mirrors.estointernet.in (mirrors.estointernet.in)|43.255.166.254|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 219929956 (210M) [application/octet-stream]
Saving to: ‘spark-3.0.1-bin-hadoop2.7.tgz’

spark-3.0.1-bin-had 100%[===================>] 209.74M  8.87MB/s    in 35s     

2020-11-04 10:30:33 (5.94 MB/s) - ‘spark-3.0.1-bin-hadoop2.7.tgz’ saved [219929956/219929956]



Extract the zip file(Spark)

In [3]:
!tar xf spark-3.0.1-bin-hadoop2.7.tgz

Install FindSpark for notebook

In [4]:
!pip install -q findspark

Set Env(path) for Java and Spark

In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"

In [6]:
#run findspark
#initialize pyspark
import findspark
findspark.init()

In [7]:
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [8]:
# create the session
conf = SparkConf()

# create the context
sc = SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

#Get the data

In [9]:
#get 20newbytes
!wget http://qwone.com/~jason/20Newsgroups/20news-18828.tar.gz

--2020-11-04 10:30:47--  http://qwone.com/~jason/20Newsgroups/20news-18828.tar.gz
Resolving qwone.com (qwone.com)... 173.48.209.137
Connecting to qwone.com (qwone.com)|173.48.209.137|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14666916 (14M) [application/x-gzip]
Saving to: ‘20news-18828.tar.gz’

20news-18828.tar.gz 100%[===================>]  13.99M  10.1MB/s    in 1.4s    

2020-11-04 10:30:48 (10.1 MB/s) - ‘20news-18828.tar.gz’ saved [14666916/14666916]



In [10]:
#extract 20newsbytes
!tar xzf 20news-18828.tar.gz

In [11]:
s = sc.wholeTextFiles('/content/20news-18828/*/*')

#Preprocess the data



In [12]:
import fileinput,re
from contextlib import redirect_stdout
import os
import nltk
from nltk.corpus import stopwords
with redirect_stdout(open(os.devnull, "w")): #suppress the output
    nltk.download('stopwords')
sw = stopwords.words('english') 

In [13]:
def remove_stopwords(text):
  filtered_words = [w for w in text.split()  if ((not w in sw) and (len(w)>1))]
  return " ".join(filtered_words)

In [14]:
from nltk.stem import SnowballStemmer
snowball = SnowballStemmer('english')
Stem1 = lambda x:snowball.stem(x)
def stem2(x):
  return ' '.join(list(map(Stem1,x.split())))

In [15]:
import string
def preprocess(x):
  x = x.lower()
  x = x.strip()
  x = x.replace("\n", " ")
  x = re.sub("[^a-z]", " ",x)
  x = remove_stopwords(x)
  x = ' '.join(list(map(Stem1,x.split())))
  return x

In [16]:
s.map(lambda x: x[1]).take(10)

["From: feustel@netcom.com (David Feustel)\nSubject: Re: We knew it would happen\n\nGod forgive me for being an American who pays taxes to a government\nthat commits atrocities like the Waco Massacre of 1993.\n-- \nDave Feustel N9MYI <feustel@netcom.com>\n\nI'm beginning to look forward to reaching the %100 allocation of taxes\nto pay for the interest on the national debt. At that point the\nfederal government will be will go out of business for lack of funds.\n",
 "From: hallam@dscomsa.desy.de (Phill Hallam-Baker)\nSubject: Re: Hallam-Baker bashes tpgers (was Re: Welcome to Police State USA)\n\n\nIn article <1993Apr22.041542.11054@a.cs.okstate.edu>, kennejs@a.cs.okstate.edu (KENNEDY JAMES SCOT) writes:\n\n|>From article <C5t9IA.6F9@dscomsa.desy.de>, by hallam@dscomsa.desy.de (Phill Hallam-Baker):\n\n|>> If anyone wants to understand the paranoid mindset of Koresh I offer you\n|>> talk.politics.guns. There you can dredge the sewers of minds so hung\n|>> up on power and ego trips that t

In [17]:
cleaned = s.map(lambda x: x[1]).map(lambda x:x.lower()).map(lambda x:x.strip()).map(lambda x :x.replace("\n", " ")).map(lambda x: re.sub("[^a-z]", " ",x)).map(remove_stopwords).map(stem2)

In [18]:
cleaned.take(10)

['feustel netcom com david feustel subject knew would happen god forgiv american pay tax govern commit atroc like waco massacr dave feustel myi feustel netcom com begin look forward reach alloc tax pay interest nation debt point feder govern go busi lack fund',
 'hallam dscomsa desi de phill hallam baker subject hallam baker bash tpger welcom polic state usa articl apr cs okstat edu kennej cs okstat edu kennedi jame scot write articl ia dscomsa desi de hallam dscomsa desi de phill hallam baker anyon want understand paranoid mindset koresh offer talk polit gun dredg sewer mind hung power ego trip bend realiti arround particular set beleif hell base ludicr claim plenti fine decent peopl peopl read post peopl paranoid peopl like hey dude make paranoid argument long ago gave argu case arm control direct instead invit peopl ask would want room full occup talk polit gun person armouri attempt enter discuss spot tea probabl gave argu case arm control direct long ago poster know talk frank cra

#Mapper

In [19]:
mapped = s.mapValues(lambda x:x.lower()).mapValues(lambda x:x.strip()).mapValues(lambda x :x.replace("\n", " ")).mapValues(lambda x: re.sub("[^a-z]", " ",x)).mapValues(remove_stopwords).mapValues(stem2).flatMapValues(lambda word:word.split(" ")).map(lambda x:(x,1))

In [20]:
mapped.take(10)

[(('file:/content/20news-18828/talk.politics.guns/54514', 'feustel'), 1),
 (('file:/content/20news-18828/talk.politics.guns/54514', 'netcom'), 1),
 (('file:/content/20news-18828/talk.politics.guns/54514', 'com'), 1),
 (('file:/content/20news-18828/talk.politics.guns/54514', 'david'), 1),
 (('file:/content/20news-18828/talk.politics.guns/54514', 'feustel'), 1),
 (('file:/content/20news-18828/talk.politics.guns/54514', 'subject'), 1),
 (('file:/content/20news-18828/talk.politics.guns/54514', 'knew'), 1),
 (('file:/content/20news-18828/talk.politics.guns/54514', 'would'), 1),
 (('file:/content/20news-18828/talk.politics.guns/54514', 'happen'), 1),
 (('file:/content/20news-18828/talk.politics.guns/54514', 'god'), 1)]

#Reducer

In [21]:
reduced = mapped.reduceByKey(lambda x, y: (x+y))

In [22]:
reduced.take(10)

[(('file:/content/20news-18828/talk.politics.guns/54514', 'netcom'), 2),
 (('file:/content/20news-18828/talk.politics.guns/54514', 'com'), 2),
 (('file:/content/20news-18828/talk.politics.guns/54514', 'david'), 1),
 (('file:/content/20news-18828/talk.politics.guns/54514', 'knew'), 1),
 (('file:/content/20news-18828/talk.politics.guns/54514', 'god'), 1),
 (('file:/content/20news-18828/talk.politics.guns/54514', 'forgiv'), 1),
 (('file:/content/20news-18828/talk.politics.guns/54514', 'american'), 1),
 (('file:/content/20news-18828/talk.politics.guns/54514', 'waco'), 1),
 (('file:/content/20news-18828/talk.politics.guns/54514', 'myi'), 1),
 (('file:/content/20news-18828/talk.politics.guns/54514', 'begin'), 1)]

#Create a dataframe 

In [23]:
df = spark.createDataFrame(reduced.map(lambda x:(x[0][0],x[0][1],x[1])))

In [24]:
df.show()

+--------------------+--------+---+
|                  _1|      _2| _3|
+--------------------+--------+---+
|file:/content/20n...|  netcom|  2|
|file:/content/20n...|     com|  2|
|file:/content/20n...|   david|  1|
|file:/content/20n...|    knew|  1|
|file:/content/20n...|     god|  1|
|file:/content/20n...|  forgiv|  1|
|file:/content/20n...|american|  1|
|file:/content/20n...|    waco|  1|
|file:/content/20n...|     myi|  1|
|file:/content/20n...|   begin|  1|
|file:/content/20n...| forward|  1|
|file:/content/20n...|   reach|  1|
|file:/content/20n...|   alloc|  1|
|file:/content/20n...|    debt|  1|
|file:/content/20n...|   feder|  1|
|file:/content/20n...|    fund|  1|
|file:/content/20n...|      de|  3|
|file:/content/20n...|   phill|  3|
|file:/content/20n...|   baker|  3|
|file:/content/20n...|    bash|  1|
+--------------------+--------+---+
only showing top 20 rows



In [25]:
columns = ['filename','word','count']
df = df.toDF(*columns)

In [26]:
df.show()

+--------------------+--------+-----+
|            filename|    word|count|
+--------------------+--------+-----+
|file:/content/20n...|  netcom|    2|
|file:/content/20n...|     com|    2|
|file:/content/20n...|   david|    1|
|file:/content/20n...|    knew|    1|
|file:/content/20n...|     god|    1|
|file:/content/20n...|  forgiv|    1|
|file:/content/20n...|american|    1|
|file:/content/20n...|    waco|    1|
|file:/content/20n...|     myi|    1|
|file:/content/20n...|   begin|    1|
|file:/content/20n...| forward|    1|
|file:/content/20n...|   reach|    1|
|file:/content/20n...|   alloc|    1|
|file:/content/20n...|    debt|    1|
|file:/content/20n...|   feder|    1|
|file:/content/20n...|    fund|    1|
|file:/content/20n...|      de|    3|
|file:/content/20n...|   phill|    3|
|file:/content/20n...|   baker|    3|
|file:/content/20n...|    bash|    1|
+--------------------+--------+-----+
only showing top 20 rows



In [27]:
from pyspark.sql import functions as func
from pyspark.sql.functions import col

create a function for highlighting

In [29]:
from termcolor import colored
def highlight(dicts, text):
  text = text.lower()
  regex = re.compile("(%s)" % "|".join( dicts.keys()))
  return regex.sub(lambda x: dicts[x.string[x.start():x.end()]], text) 

Enter a query

In [30]:
#@title What would you like to search? 
#amusing atheists and agnostics organization
original_query = 'accounts of anti-armenian human right violatins' #@param {type:"string"}
processed_query = preprocess(original_query).split()
dicts = {}
for i in original_query.lower().split():
    dicts[i] = colored(i, 'white', 'on_blue')
print(dicts)

{'accounts': '\x1b\x1baccounts\x1b', 'of': '\x1b\x1bof\x1b', 'anti-armenian': '\x1b\x1banti-armenian\x1b', 'human': '\x1b\x1bhuman\x1b', 'right': '\x1b\x1bright\x1b', 'violatins': '\x1b\x1bviolatins\x1b'}


#get the top 3 docs

In [31]:
top_3 = df.filter(df.word.isin(processed_query)).groupby('filename').agg(func.sum("count")).sort(col("sum(count)").desc()).select('filename').head(3)


In [32]:
top_3

[Row(filename='file:/content/20news-18828/talk.politics.mideast/76479'),
 Row(filename='file:/content/20news-18828/talk.politics.mideast/76392'),
 Row(filename='file:/content/20news-18828/talk.politics.mideast/77257')]

In [33]:
lines = []
for i in range(3):
  line = sc.textFile(top_3[i][0])
  lines.append(line.collect())
  

Output

In [35]:
import textwrap
for i in range(len(lines)):
  print('\n'+'='*80+'DOCUMENT '+str(i+1)+'='*80+'\n')
  print(textwrap.fill(highlight(dicts,' '.join(lines[i])),200))


================================================================================DOCUMENT 1================================================================================

from: dbd@urartu.sdpa.org (david davidian) subject: accounts of anti-armenian human right violations in azerbaijan #015
accounts of anti-armenian human right violations in azerbaijan #015                  prelude to current events in nagorno-karabakh
+-----------------------------------------------------------+          |                                                           |          | . . . they beat up the husband, dragged the wife
outside, |          | and stood her naked next to our burning things; her       |           | husband was lying at her feet on the ground. the crowd    |          | shouted, "look at the naked
armenian!" they were going to |          | throw the poor woman into the fire...mamma wouldn't allow |          | it but i went to the window and saw her standing there,   |          | and t